In [ ]:
%load_ext autoreload
%autoreload 2

import matplotlib.pyplot as plt
from renderer import SubGridSpecRenderer, GridSpecRenderer, AxesRenderer, \
PlotRenderer, render
import os
from benchart import BenchArt, Run
from metadata import RunMetadata
from loader import flatten, extract, normalize


results_dir='run_data2'
runs = []
for i, datafile in enumerate(os.listdir(results_dir)):
    all_data = extract(os.path.join(results_dir, datafile))
    data = all_data['data']['pgbench']['progress']
    metadata = all_data['metadata']
    runs.append(
        Run(i + 1, data, RunMetadata(flatten(metadata)), os.path.join(results_dir, datafile))
    )

normalize(runs)

benchart = BenchArt(runs)

relabels = {
    'machine_instance_hostinfo_Hostname': 'host',
    'benchmark_config_scale': 'scale',
    'machine_instance_hostinfo_KernelRelease': 'kernel',
    'machine_disk_block_device_settings_nr_hw_queues': '#hwq',
    'machine_disk_block_device_settings_nr_requests': 'nr_requests',
    'machine_disk_block_device_settings_queue_depth': 'queue_depth',
    'machine_disk_block_device_settings_scheduler': 'scheduler',
    'postgres.gucs.set_gucs.huge_pages': 'huge_pages',
}

figure = plt.figure(figsize=(15,40))
benchart.part(SubGridSpecRenderer(figure, relabels), 'machine_instance_hostinfo_Hostname', 
              'machine_instance_type', 'machine_disk_size_gb')

benchart.part(SubGridSpecRenderer(figure, relabels), 'benchmark_config_scale')
benchart.part(SubGridSpecRenderer(figure, relabels), 'benchmark_large_read')

benchart.ignore('machine_instance_hostinfo_KernelRelease',
                'machine_disk_block_device_settings_nr_hw_queues',
                'machine_disk_block_device_settings_nr_requests',
                'machine_disk_block_device_settings_queue_depth',
                'machine_disk_block_device_settings_scheduler',
                'postgres.gucs.set_gucs.huge_pages')

render(benchart, figure, relabels)
